In [2]:
%defaultDatasource jdbc:h2:mem:db

# Tabela Mental Health Issues

Tabela com os dados de números de pessoas, por país, com as seguintes doneças:

* depressão
* ansiedade
* bipolaridade
* problemas alimentares
* esquizofrenia.

Foi necessários fazer manipulações manuais para mudar nome de colunas e remover vírgulas para não atrapalhar na leitura do CSV.

## Porcentagem da populção com distúrbios alimentares por país

In [42]:
DROP TABLE IF EXISTS DisturbiosAlimentares;

CREATE TABLE DisturbiosAlimentares (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60) NOT NULL,
    share DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    'Distúrbios Alimentares',
    EatingDisorders
FROM CSVREAD('../data/external/share-with-an-eating-disorder.csv');

In [43]:
select * from DisturbiosAlimentares

## Porcentagem da populção com ansiedade por país

In [55]:
DROP TABLE IF EXISTS Ansiedade;

CREATE TABLE Ansiedade (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60) NOT NULL,
    share DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    'Ansiedade',
    Anxiety
FROM CSVREAD('../data/external/share-with-anxiety-disorders.csv');

In [41]:
select * from Ansiedade

## Porcentagem da populção com transtorno bipolar por país

In [44]:
DROP TABLE IF EXISTS Bipolar;

CREATE TABLE Bipolar (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60) NOT NULL,
    share DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    'Transtorno Bipolar',
    Bipolar
FROM CSVREAD('../data/external/share-with-bipolar-disorder.csv');

In [45]:
select * from Bipolar

## Porcentagem da populção com depressão por país

In [56]:
DROP TABLE IF EXISTS Depressao;

CREATE TABLE Depressao (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60) NOT NULL,
    share DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    'Depressão',
    Depressive
FROM CSVREAD('../data/external/share-with-depression.csv');

In [47]:
select * from Depressao

## Porcentagem da populção com esquizofrenia por país

In [48]:
DROP TABLE IF EXISTS Esquizofrenia;

CREATE TABLE Esquizofrenia (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60) NOT NULL,
    share DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    'Esquizofrenia',
    Schizophrenia
FROM CSVREAD('../data/external/share-of-population-with-schizophrenia.csv');

In [49]:
select * from Esquizofrenia

In [63]:
DROP TABLE IF EXISTS MentalHealthDisorders;

CREATE TABLE MentalHealthDisorders (
    country VARCHAR(60) NOT NULL,
    year INT NOT NULL,
    disease VARCHAR(60),
    share DOUBLE
) as  Select country, year, disease, share 
from Depressao
union 
Select country, year, disease, share 
from Ansiedade
union
Select country, year, disease, share 
from DisturbiosAlimentares
union
Select country, year, disease, share 
from Bipolar
union
Select country, year, disease, share 
from Esquizofrenia

In [64]:
select * from MentalHealthDisorders
order by country, year

In [31]:
DROP TABLE IF EXISTS MentalHealthIssues;

CREATE TABLE MentalHealthIssues (
    pais VARCHAR(60) NOT NULL,
    ano INT NOT NULL,
    depressao DOUBLE NOT NULL,
    ansiedade DOUBLE NOT NULL,
    bipolaridade DOUBLE NOT NULL,
    alimentar DOUBLE NOT NULL,
    esquizofrenia DOUBLE NOT NULL
) AS SELECT
    Entity,
    Year,
    Depressive,
    Bipolar,
    Anxiety,
    EatingDisorders,
    Schizophrenia
FROM CSVREAD('../data/external/mental-health-issues.csv');

In [38]:
select * from MentalHealthIssues

## Tabelas de suicídio porcetagem

In [2]:
DROP TABLE IF EXISTS ShareDeathsSuicide;

CREATE TABLE ShareDeathsSuicide (
    pais VARCHAR(60) NOT NULL,
    codigo VARCHAR(10),
    ano INT NOT NULL,
    porcentagemDeMortes DOUBLE NOT NULL
) AS SELECT
    Entity,
    Code,
    Year,
    Percent
FROM CSVREAD('../data/external/share-deaths-suicide.csv');

In [3]:
select * from ShareDeathsSuicide

## Tabelas de suicídio rates por sexo

In [3]:
DROP TABLE IF EXISTS RatesDeathsSuicide;

CREATE TABLE RatesDeathsSuicide (
    pais VARCHAR(60) NOT NULL,
    codigo VARCHAR(10),
    ano INT NOT NULL,
    RatesDeaths DOUBLE,
    RatesDeathsMale DOUBLE,
    RatesDeathsFemale DOUBLE
) AS SELECT
    Entity,
    Code,
    Year,
    Rate,
    MaleRate,
    FemaleRate  
FROM CSVREAD('../data/external/suicide-death-rates-by-sex.csv');

In [40]:
select * from RatesDeathsSuicide order by pais

## Tabelas de suicídio rates por idade

In [43]:
DROP TABLE IF EXISTS RatesDetailedDeathsSuicide;

CREATE TABLE RatesDetailedDeathsSuicide (
    pais VARCHAR(60) NOT NULL,
    codigo VARCHAR(10),
    ano INT NOT NULL,
    RatesDeaths DOUBLE,
    RatesDeaths514 DOUBLE,
    RatesDeaths1549 DOUBLE,
    RatesDeaths5069 DOUBLE,
    RatesDeaths70 DOUBLE
) AS SELECT
    Entity,
    Code,
    Year,
    Rate,
    Age514,
    Age1549,
    Age5069,
    Age70
FROM CSVREAD('../data/external/suicide-rates-by-age-detailed.csv');

In [44]:
select * from RatesDetailedDeathsSuicide order by pais

## Dúvidas:

* Não sei como colocar as partes de **Valor Absoluto** e **Valor Relativo**.
* Estou com dúvida a respeito dos valores por causa do ponto, não sei se estão certos (as vezes ele ta lendo errado, não sei).
* Pra fazer gráficos ao longo do tempo, por exemplo, como poderíamos juntar aqui com Python?
* E as chaves primárias para fazer as relações com as tabelas? (escolher duas colunas?)
* Como exportar para csv as tabelas.

# Paises

In [6]:
Select distinct RatesDeathsSuicide.pais
from RatesDeathsSuicide, MentalHealthIssues
where RatesDeathsSuicide.pais = MentalHealthIssues.pais
and RatesDeathsSuicide.ano = MentalHealthIssues.ano
order by pais

# Queries

## Top 3 doenças que mais aparecem nos países com maior taxa de suicídio.

1- Meida da taxa de mortes por suicidio para cada pais em ordem do maior para o menor

In [10]:
DROP VIEW IF EXISTS maioresTaxas;

CREATE VIEW maioresTaxas AS
select top 10 pais, AVG(RatesDeaths) as media from RatesDeathsSuicide
group by pais
order by media desc

In [11]:
select * from maioresTaxas

## Transformando colunas em linhas
### Tabela Depressão

In [52]:
DROP TABLE IF EXISTS Depressao;

CREATE TABLE Depressao (
    pais VARCHAR(60) NOT NULL,
    ano INT NOT NULL,
    disease VARCHAR(20),
    rate DOUBLE
) as  Select MentalHealthIssues.pais, MentalHealthIssues.ano, 'depressao', depressao 
from MentalHealthIssues

In [33]:
select * from Depressao

### Tabela Ansiedade

In [50]:
DROP TABLE IF EXISTS Ansiedade;

CREATE TABLE Ansiedade (
    pais VARCHAR(60) NOT NULL,
    ano INT NOT NULL,
    disease VARCHAR(20),
    rate DOUBLE
) as  Select MentalHealthIssues.pais, MentalHealthIssues.ano, 'ansiedade', ansiedade 
from MentalHealthIssues

In [15]:
select * from Ansiedade

### Juntando as doenças em uma tabela só

In [53]:
DROP TABLE IF EXISTS TodasAsDoencas;

CREATE TABLE TodasAsDoencas (
    pais VARCHAR(60) NOT NULL,
    ano INT NOT NULL,
    disease VARCHAR(20),
    rate DOUBLE
) as  Select pais, ano, disease, rate 
from Depressao
union 
Select pais, ano, disease, rate 
from Ansiedade

In [54]:
select * from TodasAsDoencas
order by pais

## Selcionado a maior doença a partir da tabela anterior transformada em linha para todadas as doenças
### Selecionando a média para cada doença

In [19]:
DROP view IF EXISTS maiorDoenca;

CREATE view maiorDoenca as
select pais, disease, AVG(rate) as mediaTaxa from TodasAsDoencas
group by pais, disease
order by pais, mediaTaxa desc

In [20]:
select * from maiorDoenca

In [21]:
select md1.pais, md1.disease, md1.mediaTaxa from maiorDoenca as md1
where md1.disease in (select top 1 md2.disease from maiorDoenca as md2
                    where md1.pais = md2.pais
                    order by mediaTaxa desc)

In [83]:
select top 1 disease from maiorDoenca
where pais = 'Afghanistan'
order by mediaTaxa desc

depressao

### Selecionando a maior média, ou seja, a doença mais prevalente

In [75]:
select pais, max(mediaTaxa) from maiorDoenca
group by pais
order by pais

## Comparando as doenças mais prevalentes por coluna

In [7]:
DROP TABLE IF EXISTS doencaPrevalente;

CREATE TABLE doencaPrevalente (
    pais VARCHAR(60) NOT NULL,
    ano INT NOT NULL,
    RatesDeaths DOUBLE,
    depressao DOUBLE NOT NULL,
    ansiedade DOUBLE NOT NULL,
    bipolaridade DOUBLE NOT NULL,
    alimentar DOUBLE NOT NULL,
    esquizofrenia DOUBLE NOT NULL,
    maior1 VARCHAR(20),
    maior2 VARCHAR(20),
    maior3 VARCHAR(20)
) as  Select RatesDeathsSuicide.pais, RatesDeathsSuicide.ano, RatesDeaths, depressao, ansiedade, bipolaridade, alimentar, esquizofrenia, null, null, null 
from RatesDeathsSuicide, MentalHealthIssues, maioresTaxas
where RatesDeathsSuicide.pais = MentalHealthIssues.pais
and RatesDeathsSuicide.ano = MentalHealthIssues.ano
and maioresTaxas.pais = RatesDeathsSuicide.pais
and RatesDeaths IS NOT NULL
order by pais

### Selcionando a primeira maior doença
* Para fazer a segunda e a terceira maior será necessário zerar o campo da mior anterior para poder comparar as colunas

In [8]:
update doencaPrevalente 
set maior1 = 'depressao'
where depressao > ansiedade
and depressao > bipolaridade
and depressao > alimentar
and depressao > esquizofrenia

In [9]:
select * from doencaPrevalente